In [102]:
import torch
from   sklearn.model_selection import train_test_split
from   sklearn.datasets import make_classification
from   sklearn.preprocessing import StandardScaler
from   sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [103]:
X, y = make_classification(
    n_samples=7000, n_features=10, n_informative=2, scale=100, random_state=32,
    n_clusters_per_class=1, hypercube=False, class_sep=10
    )

In [104]:
X.shape

(7000, 10)

In [105]:
X = StandardScaler().fit_transform(X)

In [106]:
X = torch.tensor(
    data=X, dtype=torch.float32
)
y = torch.tensor(
    data=y.reshape(-1, 1), dtype=torch.float32
)

In [107]:
xtrain, xtest, ytrain, ytest = train_test_split(
    X, y, train_size=0.25, random_state=1
)

In [119]:
from torch import nn
from torch.nn import functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.input_ = nn.Linear(in_features=10, out_features=20)
        self.hidden_1 = nn.Linear(in_features=20, out_features=10)
        self.hidden_2 = nn.Linear(in_features=10, out_features=5)
        self.hidden_3 = nn.Linear(in_features=5, out_features=2)
        self.output_  = nn.Linear(in_features=2, out_features=1)
    
    def kernel_initializer(self):
        for layer in self.modules():
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(
                    layer.weight, mode='fan_in', nonlinearity='elu'
                    )
                if layer.bias is not None:
                    nn.init.constant_(
                        layer.bias, 0.
                    )
        
    def forward(self, x):
        
        x = F.elu(self.input_(x))
        x = F.elu(self.hidden_1(x))
        x = F.elu(self.hidden_2(x))
        x = F.elu(self.hidden_3(x))
        x = F.elu(self.output_(x))
        return x
model = Model()      

In [109]:
from torch.utils.data import TensorDataset,DataLoader

In [110]:
train = TensorDataset(xtrain, ytrain)

In [111]:
train = DataLoader(
    dataset=train, 
    batch_size=32, 
    shuffle=True
    )

In [112]:
loss_fn = torch.nn.BCELoss()

optim = torch.optim.SGD(
    params=model.parameters(), momentum=0.9, nesterov=True
)

In [113]:
def power_schedule(eta_0, s=10, c=1):
    def power_fn(epoch):
        return eta_0/(1+( epoch/s ))**c
    return power_fn

power = power_schedule(eta_0=0.01, s=10, c=10)
schedule = torch.optim.lr_scheduler.LambdaLR(
    optimizer=optim, lr_lambda=power
)

In [121]:
for epoch in range(10):
    loss_mean = []
    accuracy_mean = []
    model.train()
    for xbatch, ybatch in train:

        pred = model(xbatch)
        
        pred = (pred.round()).float()
        
        
        loss = loss_fn(ybatch, pred)

        loss.backward()
        optim.step()
        optim.zero_grad()

        accuracy_mean.append(
            accuracy_score(
                y_pred=pred.detach().numpy(),
                y_true=ybatch.detach().numpy()
            )
        )
        loss_mean.append(loss.item())
    schedule.step()
    lr_i = optim.param_groups[0]['lr'] 
    string   = f''' \
    ({epoch}) loss : {np.mean(loss_mean):.3f} -- accuracy:{np.mean(accuracy_mean):.3f} -- lr:{lr_i}\
'''
    print(string)

     (0) loss : 51.173 -- accuracy:0.488 -- lr:9.765625e-09
     (1) loss : 51.276 -- accuracy:0.487 -- lr:5.995246616608976e-09
     (2) loss : 51.224 -- accuracy:0.488 -- lr:3.765071185835268e-09
     (3) loss : 51.302 -- accuracy:0.487 -- lr:2.4139131457213464e-09
     (4) loss : 51.198 -- accuracy:0.488 -- lr:1.5772029579086503e-09
     (5) loss : 51.173 -- accuracy:0.488 -- lr:1.048576e-09
     (6) loss : 51.276 -- accuracy:0.487 -- lr:7.083803738906806e-10
     (7) loss : 51.353 -- accuracy:0.486 -- lr:4.856935749618858e-10
     (8) loss : 51.198 -- accuracy:0.488 -- lr:3.3761340853132606e-10
     (9) loss : 51.250 -- accuracy:0.487 -- lr:2.3769498616783645e-10
